In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
dataset_DF = pd.read_csv("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/data/sentipolc_train_processed.csv")

In [ ]:
#Recreate the transformer
from sklearn.preprocessing import OneHotEncoder
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# define example
data = dataset_DF["opos"]
values = array(data)

# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)

# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])


In [ ]:
#Load data
import joblib
import pandas as pd
emb_matrix_test = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/creagritrend_embedd_986_128_768_alberto.joblib")
dataset_test = pd.read_csv("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/data/creagritrend_processed.csv")
wmal_matrix_test = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/support_files/creagritrend_wmal_agritrend_986_128.joblib")

In [ ]:
#Encoding data
data = dataset_test["label_pos"]
values = array(data)

# integer encode
integer_encoded_test = label_encoder.transform(values)

# binary encode
integer_encoded_test = integer_encoded_test.reshape(len(integer_encoded_test), 1)
onehot_encoded_test = onehot_encoder.transform(integer_encoded_test)


In [ ]:
!pip install keras-self-attention
from keras.layers import Input, Dense, LSTM, Bidirectional, TimeDistributed, Flatten, Concatenate, Dropout
from keras_self_attention import SeqWeightedAttention, SeqSelfAttention
from keras.layers import Add, Multiply

import keras
from sklearn.metrics import precision_recall_fscore_support as score
import numpy as np

input_emb = Input(shape=(128,768))
input_wmal = Input(shape=(128,))

#Bi-LSTM
#bi = Bidirectional(LSTM(64, return_sequences=True))(input_emb)
bi = LSTM(64, return_sequences=True)(input_emb)
att = SeqSelfAttention(attention_width=15,attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
                       attention_activation="sigmoid",
                       kernel_regularizer=keras.regularizers.l2(1e-8),#(1e-6),
                       bias_regularizer=keras.regularizers.l1(1e-4),#(1e-4),
                       attention_regularizer_weight=1e-8,#1e-4,
                       use_attention_bias=True) (bi)

app = Add() ([bi, att])                      
flat = Flatten()(app)
den1 = Dense(4000) (flat)
den2 = Dense(2000) (den1)
den3 = Dense(1000) (den2)
den4 = Dense(500) (den3)
den5 = Dense(256) (den4)
den6 = Dense(128) (den5)
den7 = Dense(64) (den6)

#WMAL 
d1 = Dense(64) (input_wmal)

#Concat
concat = Concatenate(axis=1)([den7, d1])
d1 = Dense(32, activation="relu") (concat)
output = Dense(2,activation="softmax") (d1)

m = keras.Model(inputs=[input_emb,input_wmal], outputs=output)

opt = keras.optimizers.RMSprop()

m.compile(loss="categorical_crossentropy",optimizer=opt,metrics=["accuracy"])


  Created wheel for keras-self-attention: filename=keras_self_attention-0.49.0-cp36-none-any.whl size=19468 sha256=202a1623861a94be576ae7c3d309d1592f23e54e20db78a857db12b9b8d738ff
  Stored in directory: /root/.cache/pip/wheels/6f/9d/c5/26693a5092d9313daeae94db04818fc0a2b7a48ea381989f34
Successfully built keras-self-attention


In [ ]:
m.load_weights("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/models/model_4_agritrend/model_new_pos_agritrend_0.60324.h5")

#Start explaination model


In [ ]:
tokenized_text = joblib.load("/content/drive/MyDrive/SI_AI_KBS/integrazione_AlBERTo_WMAL/sentipolc_agritrend/data/agritrend_tokenized.joblib")

In [ ]:
def calculte_CI_CU(predictions, true_label, y):
  import numpy as np
  predictions_vec = np.zeros((100,2))
  Cmax = 0.0
  Cmin = 1.0
  count = []
  i = 0
  for item in predictions:
    index = np.argmax(item);
    predictions_vec[i][index] = 1
    i = i+1
  labels_to_eval = []
  for vec, probs in zip(predictions_vec,predictions):
    max = np.max(probs)
    inverted = label_encoder.inverse_transform([argmax(vec)])[0]
    if inverted == true_label:
      count.append(1)
      if max > Cmax:
        Cmax = max
      if max < Cmin:
        Cmin = max
    labels_to_eval.append(inverted)

  coun = [1 for x in labels_to_eval if x == true_label]
  perc_correct= coun.count(1)/100
  utility = (Cmax-Cmin)
  if utility == 0:
    importance = 0
  else:
    importance = (y-Cmin)/utility

  return utility, importance


In [ ]:
#predictions = m.predict([emb_matrix_test,wmal_matrix_test])
import random
i = 0
predictions_to_eval =  m.predict([emb_matrix_test, wmal_matrix_test])

file_tokens = open("tokens.csv", "a")
file_utility = open("utility.csv", "a")
file_importances = open("importance.csv", "a")

for embedding, wmal, tokens in zip(emb_matrix_test, wmal_matrix_test, tokenized_text):
  #Dati singolo tweet
  print(tokens)
  #Per ogni token alteriamo il valore del WMAL e creiamo 1000 esempi dummy
  j = 0

  predicted = label_encoder.inverse_transform([argmax(predictions_to_eval[i])])[0]
  y = np.max(predictions_to_eval[i])

  strTokens = ""
  strUtility =""
  strImport =""
  
  for tok in tokens:
    wmals_fake = []
    embeddings_fake = []

    mean = wmal[j]
    stdev = 3   # 99.73% chance the sample will fall in your desired range
    values = [random.gauss(mean, stdev) for _ in range(100)]
    cop = wmal.copy()
    for v in values:
      cop[j] = v
      wmals_fake.append(cop.copy())
      embeddings_fake.append(embedding)

    predictions = m.predict([np.array(embeddings_fake), np.array(wmals_fake)])
    utility, importance = calculte_CI_CU(predictions,predicted, y)

    strTokens = strTokens + tok +","
    strUtility = strUtility + str(utility) +","
    strImport = strImport + str(importance) +","

    j = j+1

  strTokens = strTokens[0:-1]
  strUtility = strUtility[0:-1]
  strImport = strImport[0:-1]
  file_tokens.write(strTokens+"\n")
  file_utility.write(strUtility+"\n")
  file_importances.write(strImport+"\n")
  file_tokens.flush()
  file_utility.flush()
  file_importances.flush()
  i = i+1
file_tokens.close()
file_utility.close()
file_importances.close()

['<user>', '<hashtag>', 'riso', '</hashtag>', 'procedura', 'per', 'ripristino', 'dazi', 'al', 'rush', 'finale', 'regolamento', 'ue', 'previsto', 'per', 'il', '<number>', 'gennaio', '<url>', 'di', '<user>']
['<user>', 'gelate', 'xylella', 'sblocco', 'psr', 'a', '<hashtag>', 'bari', '</hashtag>', 'l', 'invasione', 'dei', '<hashtag>', 'gilet', 'arancio', 'ni', '</hashtag>', 'il', 'video', 'del', 'portavoce', 'spagnoletti', 'zeuli', '<url>', 'di', '<user>']
['<user>', '<user>', '<hashtag>', 'gilet', 'arancio', 'ni', '</hashtag>', 'per', 'confeuro', 'protesta', 'giusta', 'ma', 'serve', 'coerenza', 'e', 'assunzione', 'responsabilità', '<url>', 'di', '<user>']
['<user>', 'uova', 'timbratura', 'all', 'origine', 'necessaria', 'per', 'tutelare', 'i', 'consumatori', 'e', 'la', 'filiera', 'made', 'in', 'italy', '<url>', 'di', '<user>']
['<user>', '<user>', '/', 'sicurezza', 'alimentare', 'e', 'sostanze', 'chimiche', 'il', 'sistema', 'ue', 'sottoposto', 'a', 'forti', 'pressioni', 'per', 'corte', 'd